In [53]:
import numpy as np
from math import sqrt
import ot

In [49]:
def gen_4mom_approx(_w_dim: int, _batch_size: int, _W: np.ndarray = None, _K: np.ndarray = None, _H: np.ndarray = None):

    _a_dim = int(_w_dim*(_w_dim - 1)//2)

    lst = []
    for k in range(_w_dim):
        for l in range(k+1, _w_dim):
            lst.append((k,l))

    if _W is None:
        __W = np.random.randn(_batch_size, _w_dim)
    else:
        __W = _W

    if _H is None:
        __H = 1/12 * np.random.randn(_batch_size, _w_dim)
    else:
        __H = _H

    if _K is None:
        __K = 1/720 * np.random.randn(_batch_size, _w_dim)
    else:
        __K = _K

    squared_K = np.square(__K)
    C = np.random.exponential(8/15, size=(_batch_size, _w_dim))

    p = 21130/25621
    c = sqrt(1/3) - 8/15

    ber = np.random.binomial(1, p = p, size=(_batch_size, _a_dim))
    uni = np.random.uniform(-sqrt(3), sqrt(3), size=(_batch_size, _a_dim))
    rademacher = np.ones(_a_dim) - 2* np.random.binomial(1, 0.5, size= (_batch_size, _a_dim))
    ksi = ber*uni + (1-ber)*rademacher
    #print(ksi)
    def sigma(i: int, j: int):
        return np.sqrt(3/28*(C[:,i] + c)*(C[:,j] + c) + 144/28*(squared_K[:,i] + squared_K[:,j]))

    idx = 0
    for k in range(_w_dim):
        for l in range(k+1, _w_dim):
            sig = sigma(k, l)
            #print(f"shape: {sig.shape}, k: {k}, l: {l}, sig: {sig}")

            # now calculate a from ksi and sigma (but store a in ksi)
            ksi[:, idx] *= sig

            # calculate the whole thing
            ksi[:, idx] += __H[:,k]*__W[:,l] - __W[:,k]*__H[:,l] + 12*(__K[:,k]*__H[:,l] - __H[:,k]*__K[:,l])
            idx += 1

    return ksi

print(gen_4mom_approx(3, 5))

[[ 0.21319312  0.10747754 -0.21131219]
 [ 0.18869785 -0.0516751  -0.07077597]
 [-0.07774102 -0.21811654 -0.37720795]
 [-0.00880003 -0.06745311  0.17446786]
 [ 0.03712373  0.01866568 -0.19683424]]


In [50]:
samples = np.genfromtxt('samples/fixed_samples_3-dim.csv', dtype=np.float, delimiter=',')
W = samples[:,:3]
a_true = samples[:,3:6]
print(samples.shape)
bsz = samples.shape[0]
generated = gen_4mom_approx(3,bsz,_W=W)


/tmp/ipykernel_11558/40691991.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  samples = np.genfromtxt('samples/fixed_samples_3-dim.csv', dtype=np.float, delimiter=',')


(65536, 6)


In [51]:
print(generated.shape)

(65536, 3)


In [54]:
err = [sqrt(ot.wasserstein_1d(a_true[:,i],generated[:,i],p=2)) for i in range(3)]
print(err)

[0.22306273091689732, 0.30748067361934256, 0.2555661935985148]
